In [3]:
from sleeper_wrapper.league import League
from sleeper_wrapper.base_api import BaseApi
from sleeper_wrapper.user import User
from sleeper_wrapper.drafts import Drafts
from sleeper_wrapper.stats import Stats
from sleeper_wrapper.players import Players
import pandas as pd
import json
import pprint
import matplotlib.pyplot as plt

In [14]:
league_21 = League(719361504427728896)
league_22 = League(784689960694439936)
league_23 = League(959512549940088832)

leagues = [league_21, league_22, league_23]

In [16]:
users_21 = league_21.get_users()
users_22 = league_22.get_users()
users_23 = league_23.get_users()

users = [users_21, users_22, users_23]

In [47]:
display_names_to_user_ids = {}
for object in users:
    for user in object:
        display_names_to_user_ids[user['display_name']] = user['user_id']


In [70]:
pprint.pp(display_names_to_user_ids)

{'Perrry': '712402115632373760',
 'Termie': '712402564943028224',
 'Mikmikcrf': '713281083025747968',
 'Swooboobler': '713589650459033600',
 'roco5marco': '713592138331422720',
 'chuckn0rris': '716723709183868928',
 'jamesgb': '719330870766411776',
 'CrustyPoots': '719344483250155520',
 'WuWei': '719375265469362176',
 'Ravenempire': '719404260776828928',
 'Cougar4life66': '720018737989775360',
 'gogglespaysano': '720076464837447680',
 'Nosy': '740776088590004224'}


In [24]:
del display_names_to_user_ids['Nosy']

In [35]:
rosters_21 = league_21.get_rosters()
rosters_22 = league_22.get_rosters()
rosters_23 = league_23.get_rosters()

rosters = [rosters_21, rosters_22, rosters_23]


In [32]:
all_years_matchups = []
for league_num in range(21, 24):
    league = globals()[f'league_{league_num}']

    league_year_matchups = []
    for i in range(15):
        matchups = league.get_matchups(i)
        league_year_matchups.append(matchups)

    all_years_matchups.append(league_year_matchups)

In [54]:
team_name_to_display_name = {}
for object in users:
    for user in object:
        team_name = user.get('metadata', {}).get('team_name')
        if team_name:
            team_name_to_display_name[team_name] = user['display_name']

In [81]:
roster_id_to_display_name = {}

for object in rosters:
    for roster in object:
        roster_id_to_display_name[roster['roster_id']] = roster['owner_id']

In [82]:
pprint.pp(roster_id_to_display_name)

{1: '712402115632373760',
 2: '712402564943028224',
 3: '719344483250155520',
 4: '740776088590004224',
 5: '719330870766411776',
 6: '719375265469362176',
 7: '719404260776828928',
 8: '716723709183868928',
 9: '713592138331422720',
 10: '713589650459033600',
 11: '720018737989775360',
 12: '720076464837447680'}


In [83]:
#reverse display name to user id
user_id_to_display_name = {user_id: display_name for display_name, user_id in display_names_to_user_ids.items()}

In [114]:
roster_to_display_name = {roster_id: user_id_to_display_name[user_id] for roster_id, user_id in roster_id_to_display_name.items()}


In [117]:
pprint.pp(roster_to_display_name)

{1: 'Perrry',
 2: 'Termie',
 3: 'CrustyPoots',
 4: 'Mikmikcrf',
 5: 'jamesgb',
 6: 'WuWei',
 7: 'Ravenempire',
 8: 'chuckn0rris',
 9: 'roco5marco',
 10: 'Swooboobler',
 11: 'Cougar4life66',
 12: 'gogglespaysano'}


In [116]:
roster_to_display_name[4] = 'Mikmikcrf'

In [86]:
# For leagues 21 and 22, change Nosy to Mikmikcrf

In [88]:
from collections import defaultdict

In [124]:
player_records = defaultdict(lambda: defaultdict(lambda: [0, 0]))
year_index = 0

for year_index, matchup_yr in enumerate(all_years_matchups):  # Assuming this loop goes over the years
    # Set 'Mikmikcrf' for all years except when year_index == 2
    if year_index == 2:
        roster_to_display_name[4] = 'Nosy'  # Only set 'Nosy' for year_index 2
    else:
        roster_to_display_name[4] = 'Mikmikcrf'  # For all other years, it's 'Mikmikcrf'
    
    for matchup_week in matchup_yr:
        matchup_groups = defaultdict(list)

        # Step 1: Group matchups by matchup_id
        for matchup in matchup_week:
            matchup_groups[matchup['matchup_id']].append(matchup)

        # Step 2: Process each matchup within the week
        for matchup_id, players in matchup_groups.items():
            if len(players) == 2:  # Ensure there are two players in the matchup
                player1, player2 = players

                # Get roster IDs and points
                roster_id1, points1 = player1['roster_id'], player1['points']
                roster_id2, points2 = player2['roster_id'], player2['points']
                
                # Map roster IDs to display names
                player1_name = roster_to_display_name[roster_id1]
                player2_name = roster_to_display_name[roster_id2]
                
                # Determine the winner and loser
                if points1 > points2:
                    winner, loser = player1_name, player2_name
                else:
                    winner, loser = player2_name, player1_name
                
                # Step 3: Track the win/loss record
                player_records[winner][loser][0] += 1  # Increment winner's win count against loser
                player_records[loser][winner][1] += 1  # Increment loser's loss count against winner

In [125]:
for player, record in player_records.items():
    print(f"{player}'s record against others:")
    for opponent, score in record.items():
        print(f"  vs {opponent}: {score}")

Cougar4life66's record against others:
  vs Perrry: [4, 2]
  vs CrustyPoots: [2, 2]
  vs chuckn0rris: [3, 2]
  vs jamesgb: [0, 3]
  vs Mikmikcrf: [1, 1]
  vs WuWei: [2, 1]
  vs roco5marco: [0, 4]
  vs Termie: [3, 1]
  vs Ravenempire: [1, 2]
  vs gogglespaysano: [3, 0]
  vs Swooboobler: [0, 3]
  vs Nosy: [2, 0]
Perrry's record against others:
  vs Cougar4life66: [2, 4]
  vs chuckn0rris: [4, 1]
  vs CrustyPoots: [2, 2]
  vs Mikmikcrf: [2, 0]
  vs jamesgb: [1, 2]
  vs roco5marco: [1, 3]
  vs WuWei: [3, 0]
  vs Ravenempire: [3, 0]
  vs Termie: [2, 2]
  vs Swooboobler: [2, 1]
  vs gogglespaysano: [2, 1]
  vs Nosy: [0, 2]
Termie's record against others:
  vs Ravenempire: [2, 3]
  vs WuWei: [2, 2]
  vs roco5marco: [2, 2]
  vs CrustyPoots: [2, 2]
  vs chuckn0rris: [2, 2]
  vs gogglespaysano: [2, 1]
  vs Swooboobler: [3, 1]
  vs Cougar4life66: [1, 3]
  vs Perrry: [2, 2]
  vs jamesgb: [2, 1]
  vs Mikmikcrf: [2, 0]
  vs Nosy: [1, 0]
Ravenempire's record against others:
  vs Termie: [3, 2]
  vs ro

In [126]:
pprint.pp(player_records)

defaultdict(<function <lambda> at 0x11acde660>,
            {'Cougar4life66': defaultdict(<function <lambda>.<locals>.<lambda> at 0x11acdf2e0>,
                                          {'Perrry': [4, 2],
                                           'CrustyPoots': [2, 2],
                                           'chuckn0rris': [3, 2],
                                           'jamesgb': [0, 3],
                                           'Mikmikcrf': [1, 1],
                                           'WuWei': [2, 1],
                                           'roco5marco': [0, 4],
                                           'Termie': [3, 1],
                                           'Ravenempire': [1, 2],
                                           'gogglespaysano': [3, 0],
                                           'Swooboobler': [0, 3],
                                           'Nosy': [2, 0]}),
             'Perrry': defaultdict(<function <lambda>.<locals>.<lambda> at 0x11acdc360>,


In [58]:
'''for year_data in all_years_scoreboards:
    # Skip any None or empty lists
    if not year_data:
        continue
    
    for week_data in year_data:
        # Check if week_data is a dictionary of matchups
        if isinstance(week_data, dict):
            for week, matchups in week_data.items():
                updated_matchups = []
                for team, score in matchups:
                    # Replace team name if it exists in the mapping
                    new_team_name = team_name_to_display_name.get(team, team)  # Default to the original name if no mapping
                    updated_matchups.append((new_team_name, score))
                
                # Update the matchups with the updated team names
                week_data[week] = updated_matchups'''

In [60]:
'''# Filter out any user(s) you don't want
for year_data in all_years_scoreboards:
    if not year_data:
        continue
    
    for week_data in year_data:
        if isinstance(week_data, dict):
            for week, matchups in week_data.items():
                filtered_matchups = [(team, score) for team, score in matchups if 'Nosy' not in team]
                
                # Update the week_data with the filtered matchups
                week_data[week] = filtered_matchups'''